In [1]:
import pandas as pd 
import numpy as py

In [2]:
df = pd.read_csv("0.csv")
df

,Title,Tag,Review,Address,Mention,Comment,Country,Price,Rating,tags,Governorate,img_link
0,Adventure Alegria ASEG Travel,Tours,14.0,"29, Cite des PTT, Hydra, Algiers, Algiers, Alg...",10 mentions ofÃÂ Algeria,Visiting Algeria is a truly remarkable experie...,Algeria,33.56,1.06,tours,Mostaganem,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,Fancyellow Travel Services,Tours,151.0,"Algiers, Algiers Province, Algeria",72 mentions ofÃÂ Algeria,that went above and beyond to provide us with ...,Algeria,52.07,1.60,tours,Ghardaia,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,Memorial du Martyr,Monuments & Statues,529.0,"Boulevard Khalifa Oulmane, Algiers, Algiers Pr...",31 mentions ofÃÂ Algeria,and very close to the shopping mall where you ...,Algeria,103.15,3.12,archaeological tourism,Tebessa,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,Cederberg Wilderness Area,for fun,86.0,"Clanwilliam, Western Cape, South Africa",12 mentions ofÃÂ Algeria,So instead we did a couple half day hikes arou...,Algeria,43.29,1.34,for fun,Illizi,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,Timgad,Ancient Ruins,100.0,"Timgad, Batna Province, Algeria",12 mentions ofÃÂ Algeria,Timgad was my main reason to travel to Algeria...,Algeria,45.18,1.40,archaeological tourism,El Oued,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
...,...,...,...,...,...,...,...,...,...,...,...,...
28358,Palm Beach Palace,Hotels,222.0,"Route Touristique, Tozeur, Tozeur Governorate,...",4 mentions ofÃÂ Tunisia,"Like many hotels in Tunisia, this one is suffe...",Tunisia,61.67,1.89,hotel,Medenine,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
28359,Hotel Lac Leman,Hotels,43.0,"Rue Lac Leman, Tunis, Tunis Governorate, Tunisia",2 mentions ofÃÂ Tunisia,This is not the kind of hotels you should book...,Tunisia,37.48,1.17,hotel,Monastir,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
28360,Hotel Riad Meninx Djerba,Hotels,69.0,"Zone Touristique, Midoun, Djerba Island, Meden...",Unknown,No Comment,Tunisia,40.99,1.28,hotel,Ariana,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
28361,Floriana Meninx Settemari Club,Hotels,7.0,"Djerba Island, Medenine Governorate, Tunisia",Unknown,No Comment,Tunisia,32.61,1.03,hotel,Zaghouan,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Assuming df is your DataFrame with selected columns and cleaned data

selected_columns = ['Title', 'Tag', 'Review', 'Comment', 'Country', 'Price', 'Rating', 'tags']
df = df[selected_columns].dropna()

df['Tag'] = df['Tag'].astype(str)
df['Review'] = df['Review'].astype(str)
df['Comment'] = df['Comment'].astype(str)

# Feature Engineering: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Tag'] + ' ' + df['Review'] + ' ' + df['Comment'])

# Define high and medium ratings thresholds
HIGH_RATING_THRESHOLD = 4.0
MEDIUM_RATING_THRESHOLD = 3.0

# Function to get recommendations based on chosen tags
def get_recommendations_by_tags(chosen_tags, df, num_recommendations=5):
    recommendations_df = pd.DataFrame(columns=['Title', 'Price', 'Rating', 'Country', 'tags', 'combined_text'])

    for tag in chosen_tags:
        response_indices = [
            i for i, tag_value in enumerate(df['tags']) 
            if tag.lower() in tag_value.lower() and df['Rating'].iloc[i] >= MEDIUM_RATING_THRESHOLD
        ]

        if response_indices:
            selected_indices = random.sample(response_indices, min(len(response_indices), num_recommendations))
            recommendations = df.iloc[selected_indices][['Title', 'Price', 'Rating', 'Country', 'tags']]
            recommendations['combined_text'] = df.iloc[selected_indices]['Tag'] + ' ' + df.iloc[selected_indices]['Review'] + ' ' + df.iloc[selected_indices]['Comment']
            recommendations_df = pd.concat([recommendations_df, recommendations])

    recommendations_df = recommendations_df.drop_duplicates().reset_index(drop=True)
    return recommendations_df

# Function to evaluate performance using cosine similarity
def evaluate_performance(recommendations, actual_data, threshold=0.5):
    recommendations['combined_text'] = recommendations['combined_text'].str.lower().str.strip()
    actual_data['combined_text'] = (actual_data['Tag'] + ' ' + actual_data['Review'] + ' ' + actual_data['Comment']).str.lower().str.strip()

    tfidf_actual_data = tfidf_vectorizer.transform(actual_data['combined_text'])
    tfidf_recommendations = tfidf_vectorizer.transform(recommendations['combined_text'])

    cosine_similarities = cosine_similarity(tfidf_recommendations, tfidf_actual_data)
    true_positive = sum(cosine_similarities.max(axis=1) >= threshold)
    precision = true_positive / len(recommendations) if len(recommendations) > 0 else 0
    recall = true_positive / len(actual_data) if len(actual_data) > 0 else 0
    
    print(f"\nPerformance Metrics:")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

    return precision, recall

# Example: Get recommendations based on user-chosen tags
user_chosen_tags = ["Museums", "water places", "for fun"]
recommendations = get_recommendations_by_tags(user_chosen_tags, df)

# Evaluate the model's performance with actual data from 'df'
precision, recall = evaluate_performance(recommendations, df, threshold=0.2)  # Try adjusting the threshold

# Remove recommendations if both precision and recall are 0
if precision == 0 and recall == 0:
    print("\nRemoving recommendations with precision and recall both equal to 0.")
    recommendations = pd.DataFrame(columns=['Title', 'Price', 'Rating', 'Country', 'tags', 'combined_text'])

# Display only Title, Country, and tags
print(f"\nOverall Recommendations with chosen tags {user_chosen_tags}:")
print(recommendations[['Title', 'Country', 'tags']])



Performance Metrics:
Precision: 1.00
Recall: 0.00

Overall Recommendations with chosen tags ['Museums', 'water places', 'for fun']:
                                     Title               Country          tags
0                 Veterans Memorial Museum                  Iraq       museums
1                          Al-Majid Mosque                 Egypt       museums
2   Vabamu Museum Of Occupations & Freedom                Saudi        museums
3                  El-Alamein War Cemetery                 Egypt       museums
4                          King Tut Museum                 Egypt       museums
5                             Port Beaches                 Egypt  water places
6                         Nautilus Red Sea               Bahrain  water places
7                              Gleem Beach                 Egypt  water places
8                         Terrazzina Beach                Saudi   water places
9                              Petite Anse               Bahrain  water place